## About the Dataset
1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks the article as potentially unreliable

```
1: Fake News
0: Real news
```

[Dataset](https://www.kaggle.com/c/fake-news/data?select=train.csv)

### Importing the dependency

In [4]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [35]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amitthapa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
# printing the stopwords in english
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Data Pre-processing

In [7]:
# loading the dataset to a pandas Dataframe

news_dataset = pd.read_csv("dataset/fake_news.csv")

In [11]:
news_dataset.shape

(20800, 5)

In [12]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [15]:
# replacing the null values with empty strings
news_dataset = news_dataset.fillna("")

In [17]:
# merging the author name and news title

news_dataset["content"] = news_dataset.author + " " + news_dataset["title"]

In [18]:
news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [19]:
# seperating the data and label
X = news_dataset.drop(columns=["label"], axis=1)
y = news_dataset["label"]

In [20]:
X.head()

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...


## Stemming:

Stemming is the process of reducing a word to it's root word

Example:
actor, actress, acting ---> act

In [21]:
port_stem = PorterStemmer()

In [39]:
def stemming(content):
    stemmed_content = re.sub(r"[^a-zA-Z]", " ", content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words("english")]
    stemmed_content = " ".join(stemmed_content)
    return stemmed_content

In [42]:
news_dataset["content"] = news_dataset["content"].apply(stemming)

In [43]:
print(news_dataset["content"])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [44]:
# seperating the data and the label
X = news_dataset["content"].values
y = news_dataset["label"].values

## Splitting data

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.2, stratify=y, random_state=2)

In [53]:
# converting textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)

TfidfVectorizer()

In [54]:
X_train = vectorizer.transform(X_train)

In [55]:
X_test = vectorizer.transform(X_test)

In [59]:
print(X_train)

  (0, 7383)	0.10303087652134682
  (0, 6487)	0.3788875668370965
  (0, 4851)	0.34584142373405025
  (0, 4616)	0.34022605781960225
  (0, 3952)	0.3788875668370965
  (0, 3456)	0.3595568123283494
  (0, 3114)	0.3522076005478322
  (0, 1776)	0.24381268485675361
  (0, 596)	0.3926029554313957
  (1, 4312)	0.6899178748532425
  (1, 3924)	0.7238876473307068
  (2, 7124)	0.22140398638399422
  (2, 6797)	0.32922380057444495
  (2, 6776)	0.2549978280635635
  (2, 6426)	0.18565117139670295
  (2, 5740)	0.30050927473889655
  (2, 5045)	0.3124268797346603
  (2, 4206)	0.2646756279132291
  (2, 4160)	0.25330672370768403
  (2, 3757)	0.35793832640999335
  (2, 3694)	0.2405687690530664
  (2, 2362)	0.3199797879766129
  (2, 2205)	0.2773264609710163
  (2, 831)	0.23596110207768065
  (3, 7809)	0.37331382263180085
  :	:
  (4157, 6537)	0.5838332583251654
  (4157, 6098)	0.19588097487082282
  (4157, 5859)	0.25928335675446157
  (4157, 5265)	0.28293693627343575
  (4157, 3476)	0.22615631402428993
  (4157, 3189)	0.2444854583890062
 

## Model training

In [60]:
model = LogisticRegression()

In [61]:
model.fit(X_train, y_train)

LogisticRegression()

## Model evaluation

In [62]:
y_train_prediction = model.predict(X_train)

accuracy_score(y_train, y_train_prediction)

0.9771634615384616

In [63]:
y_test_prediction = model.predict(X_test)

accuracy_score(y_test, y_test_prediction)

0.9541466346153846

## Predective system

In [67]:
X_new = X_test[1]

prediction = model.predict(X_new)
print(prediction)

if prediction[0] == 0:
    print("News is real")
else:
    print("News is fake")

[0]
News is real


In [68]:
y_test[1]

0